In [ ]:
import cv2
import os, sys, gc
import time
import numpy as np
import mediapipe as mp
from tqdm.auto import tqdm
import multiprocessing
from joblib import Parallel, delayed
from natsort import natsorted
from glob import glob
import math
import pickle

In [ ]:
mp_holistic = mp.solutions.holistic

N_FACE_LANDMARKS = 468
N_BODY_LANDMARKS = 33
N_HAND_LANDMARKS = 21

In [ ]:
class Counter(object):
    # https://stackoverflow.com/a/47562583/
    def __init__(self, initval=0):
        self.val = multiprocessing.RawValue("i", initval)
        self.lock = multiprocessing.Lock()

    def increment(self):
        with self.lock:
            self.val.value += 1

    @property
    def value(self):
        return self.val.value

In [ ]:
def process_body_landmarks(component, n_points):
    kps = np.zeros((n_points, 3))
    conf = np.zeros(n_points)
    if component is not None:
        landmarks = component.landmark
        kps = np.array([[p.x, p.y, p.z] for p in landmarks])
        conf = np.array([p.visibility for p in landmarks])
    return kps, conf

In [ ]:
def process_other_landmarks(component, n_points):
    kps = np.zeros((n_points, 3))
    conf = np.zeros(n_points)
    if component is not None:
        landmarks = component.landmark
        kps = np.array([[p.x, p.y, p.z] for p in landmarks])
        conf = np.ones(n_points)
    return kps, conf

In [ ]:
def get_holistic_keypoints(
    frames, holistic=mp_holistic.Holistic(static_image_mode=False, model_complexity=2)
):
    """
    For videos, it's optimal to create with `static_image_mode=False` for each video.
    https://google.github.io/mediapipe/solutions/holistic.html#static_image_mode
    """

    keypoints = []
    confs = []

    for frame in frames:
        results = holistic.process(frame)

        body_data, body_conf = process_body_landmarks(
            results.pose_landmarks, N_BODY_LANDMARKS
        )
        face_data, face_conf = process_other_landmarks(
            results.face_landmarks, N_FACE_LANDMARKS
        )
        lh_data, lh_conf = process_other_landmarks(
            results.left_hand_landmarks, N_HAND_LANDMARKS
        )
        rh_data, rh_conf = process_other_landmarks(
            results.right_hand_landmarks, N_HAND_LANDMARKS
        )

        data = np.concatenate([body_data, face_data, lh_data, rh_data])
        conf = np.concatenate([body_conf, face_conf, lh_conf, rh_conf])

        keypoints.append(data)
        confs.append(conf)

    # TODO: Reuse the same object when this issue is fixed: https://github.com/google/mediapipe/issues/2152
    holistic.reset()
    del holistic
    gc.collect()

    keypoints = np.stack(keypoints)
    confs = np.stack(confs)
    return keypoints, confs


In [ ]:
def gen_keypoints_for_frames(frames, save_path):

    pose_kps, pose_confs = get_holistic_keypoints(frames)
    body_kps = np.concatenate([pose_kps[:, :33, :], pose_kps[:, 501:, :]], axis=1)

    confs = np.concatenate([pose_confs[:, :33], pose_confs[:, 501:]], axis=1)

    d = {"keypoints": body_kps, "confidences": confs}

    with open(save_path + ".pkl", "wb") as f:
        pickle.dump(d, f, protocol=4)

In [ ]:
def load_frames_from_video(video_path):
    frames = []
    vidcap = cv2.VideoCapture(video_path)
    while vidcap.isOpened():
        success, img = vidcap.read()
        if not success:
            break
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # img = cv2.resize(img, (640, 480))
        frames.append(img)

    vidcap.release()
    # cv2.destroyAllWindows()
    return np.asarray(frames)

In [ ]:
def load_frames_from_folder(frames_folder, patterns=["*.jpg"]):
    images = []
    for pattern in patterns:
        images.extend(glob(f"{frames_folder}/{pattern}"))
    images = natsorted(list(set(images)))  # remove dupes
    if not images:
        exit(f"ERROR: No frames in folder: {frames_folder}")

    frames = []
    for img_path in images:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frames.append(img)

    return np.asarray(frames)

In [ ]:
def gen_keypoints_for_video(video_path, save_path):
    if not os.path.isfile(video_path):
        print("SKIPPING MISSING FILE:", video_path)
        return
    frames = load_frames_from_video(video_path)
    gen_keypoints_for_frames(frames, save_path)

In [ ]:
def gen_keypoints_for_folder(folder, save_path, file_patterns):
    frames = load_frames_from_folder(folder, file_patterns)
    gen_keypoints_for_frames(frames, save_path)

In [ ]:
def generate_pose(dataset, save_folder, worker_index, num_workers, counter):
    num_splits = math.ceil(len(dataset) / num_workers)
    end_index = min((worker_index + 1) * num_splits, len(dataset))
    for index in range(worker_index * num_splits, end_index):
        imgs, label, video_id = dataset.read_data(index)
        save_path = os.path.join(save_folder, video_id)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        gen_keypoints_for_frames(imgs, save_path)
        counter.increment()

In [ ]:
def dump_pose_for_dataset(
    dataset, save_folder, num_workers=multiprocessing.cpu_count()
):
    os.makedirs(save_folder, exist_ok=True)
    processes = []
    counter = Counter()
    for i in tqdm(range(num_workers), desc="Creating sub-processes..."):
        p = multiprocessing.Process(
            target=generate_pose, args=(dataset, save_folder, i, num_workers, counter)
        )
        p.start()
        processes.append(p)

    total_samples = len(dataset)
    with tqdm(total=total_samples) as pbar:
        while counter.value < total_samples:
            pbar.update(counter.value - pbar.n)
            time.sleep(2)

    for i in range(num_workers):
        processes[i].join()
    print(f"Pose data successfully saved to: {save_folder}")

In [ ]:
n_cores = multiprocessing.cpu_count()

Top_DIR = "archive"
SAVE_DIR = "holistic_poses"
for DIR in os.listdir(Top_DIR):
    os.makedirs(os.path.join(SAVE_DIR,DIR), exist_ok=True)

    file_paths = []
    save_paths = []
    for file in os.listdir(os.path.join(Top_DIR,DIR)):
        file_paths.append(os.path.join(Top_DIR,DIR, file))
        save_paths.append(os.path.join(SAVE_DIR,DIR, file.replace(".mp4", "")))

    
    for path, save_path in tqdm(zip(file_paths, save_paths)):
        gen_keypoints_for_video(path, save_path)

In [ ]:
mkdir output

In [ ]:
d = {"001":"adress","002":"and","003":"anise","004":"banana","005":"berell","006":"birth certificate","007":"cinnamon","008":"coffee","009":"cold","010":"deaf","011":"doing","012":"drink","013":"fenugreek","014":"fine","015":"glass","016":"hello","017":"help","018":"hot","019":"How are you","020":"How much","021":"in","022":"juice","023":"lemon","024":"mango","025":"me","026":"milk","027":"mint","028":"money","029":"my","030":"name","031":"need","032":"nescafe","033":"new","034":"no","035":"number","036":"ok","037":"on","038":"or","039":"orange","040":"pepsi","041":"problem","042":"question","043":"Roselle","044":"spoon","045":"strawberry","046":"tea","047":"Thank god","048":"wait","049":"water","050":"what","051":"yes","052":"you","053":"your","054":"Your health is fine"}

In [ ]:
swapped_dict = {value: key for key, value in d.items()}

print(swapped_dict)

In [ ]:
import os 
import shutil
data = "/kaggle/input/holistic/holistic_poses/holistic_poses"
for word in os.listdir(data):
    id = swapped_dict.get(word)
    if id:
        print(word,id)
        i=0
        for pos in os.listdir(os.path.join(data,word)):
            if i<=70:
                shutil.copy(os.path.join(data,word,pos),"output/"+id+"_001_0"+str(i)+".pkl")
                i+=1
            elif i <90:
                shutil.copy(os.path.join(data,word,pos),"output/"+id+"_009_0"+str(i)+".pkl")
                i+=1
            else:
                shutil.copy(os.path.join(data,word,pos),"output/"+id+"_010_0"+str(i)+".pkl")
                i+=1
    else:
        continue

In [ ]:
def init_model():
    !pip install openhands
    !rm /kaggle/working/OpenHands/openhands/models/ssl/dpc_rnn.py
    !cp /kaggle/input/holistic/dpc_rnn.py /kaggle/working/OpenHands/openhands/models/ssl/
    !rm /kaggle/working/OpenHands/openhands/core/exp_utils.py
    !cp /kaggle/input/holistic/exp_utils.py /kaggle/working/OpenHands/openhands/core/
    !rm /kaggle/working/OpenHands/openhands/apis/classification_model.py
    !cp /kaggle/input/test-data/classification_model.py /kaggle/working/OpenHands/openhands/apis/
    !rm /kaggle/working/OpenHands/openhands/apis/inference.py
    !cp /kaggle/input/test-data/inference.py /kaggle/working/OpenHands/openhands/apis/
    %cd OpenHands/
    !python setup.py install
    %cd ..

In [ ]:
!pip install git+https://github.com/PytorchLightning/lightning-bolts.git@master --upgrade
!git clone https://github.com/AI4Bharat/OpenHands.git
init_model()
init_model()

In [1]:
cd ..

E:\project_material


C:\Users\20155\anaconda3\envs\GP\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
import openhands
print(openhands.__version__)

0.1.4.2


In [ ]:
from omegaconf import OmegaConf
import openhands
from openhands.apis.classification_model import ClassificationModel
from openhands.core.exp_utils import get_trainer

cfg = OmegaConf.load("/kaggle/input/test-data/decoupled_gcn.yaml")
trainer = get_trainer(cfg)

model = ClassificationModel(cfg=cfg, trainer=trainer)
model.init_from_checkpoint_if_available()
model.fit()

In [ ]:
from IPython.display import FileLink
FileLink(r'best_model.zip')

In [ ]:
!zip best_model.zip /kaggle/working/experiments/lsa64/sl_gcn/epoch=47-step=1776.ckpt

In [3]:
cfg.data.test_pipeline.dataset.root_dir='Model/temp/'

In [2]:
from omegaconf import OmegaConf
from openhands.apis.inference import InferenceModel
import openhands

cfg = OmegaConf.load("Model/decoupled_gcn.yaml")
model = InferenceModel(cfg=cfg)
model.init_from_checkpoint_if_available()
if cfg.data.test_pipeline.dataset.inference_mode:
   pred = model.test_inference()
else:
    model.compute_test_accuracy()

C:\Users\20155\anaconda3\envs\GP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found 53 classes in test splits
Loading checkpoint from: E:/project_material/Model/epoch91.ckpt
Thank god:	Model/test\01_0001_(10_03_21_20_37_17)_c.pkl
Thank god:	Model/test\01_0002_(10_03_21_20_53_42)_c.pkl
help:	Model/test\VID-20240228-WA0009.pkl
birth certificate:	Model/test\VID-20240228-WA0011.pkl
How much:	Model/test\VID20230309141046.pkl
adress:	Model/test\WIN_20240301_15_47_18_Pro.pkl
How much:	Model/test\WIN_20240301_15_57_57_Pro.pkl
Avg time per iteration: 956.0108184814453 ms


In [ ]:
cfg = OmegaConf.load("/kaggle/input/test-data/decoupled_gcn.yaml")
model = InferenceModel(cfg=cfg)
model.init_from_checkpoint_if_available()
model.compute_test_accuracy()